# Evaluate Mixtral using SageMaker Clarify

See https://github.com/aws/fmeval/tree/main

In [ ]:
!pip install -U sagemaker

In [5]:
!pip install -U ipywidgets

  Obtaining dependency information for ipywidgets from https://files.pythonhosted.org/packages/4a/0e/57ed498fafbc60419a9332d872e929879ceba2d73cb11d284d7112472b3e/ipywidgets-8.1.1-py3-none-any.whl.metadata
  Obtaining dependency information for widgetsnbextension~=4.0.9 from https://files.pythonhosted.org/packages/29/03/107d96077c4befed191f7ad1a12c7b52a8f9d2778a5836d59f9855c105f6/widgetsnbextension-4.0.9-py3-none-any.whl.metadata
  Obtaining dependency information for jupyterlab-widgets~=3.0.9 from https://files.pythonhosted.org/packages/e8/05/0ebab152288693b5ec7b339aab857362947031143b282853b4c2dd4b5b40/jupyterlab_widgets-3.0.9-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 1.4 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.9/214.9 kB 2.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 16.1 MB/s eta 0:00:0000:01
  Attempting uninstall: widgetsnbextension
    Found existing installation: w

In [3]:
!pip install fmeval

  Obtaining dependency information for fmeval from https://files.pythonhosted.org/packages/7c/57/a5e7f98fb53e4871ca3ec91b687e9c4046802f8278cdc1c13a4fcf7193db/fmeval-0.2.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 811.7 kB/s eta 0:00:00 0:00:01
  Obtaining dependency information for evaluate<0.5.0,>=0.4.0 from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for ipykernel<7.0.0,>=6.26.0 from https://files.pythonhosted.org/packages/73/14/ec9a1c42d2c952c848a1ac366718783ab725148faf910bc086601809cefd/ipykernel-6.27.1-py3-none-any.whl.metadata
  Obtaining dependency information for jiwer<4.0.0,>=3.0.3 from https://files.pythonhosted.org/packages/0d/4f/ee537ab20144811dd99321735ff92ef2b3a3230b77ed7454bed4c44d21fc/jiwer-3.0.3-py3-none-any.whl.metadata
  Obtaining dependency information for matplotlib<4.0.0,>=3.8.0 from ht

In [15]:
!rm -Rf ~/.cache/pip/*

In [23]:
from fmeval.data_loaders.data_config import DataConfig
from fmeval.model_runners.sm_model_runner import SageMakerModelRunner
from fmeval.constants import MIME_TYPE_JSONLINES
from fmeval.eval_algorithms.factual_knowledge import FactualKnowledge, FactualKnowledgeConfig
from fmeval.eval import get_eval_algorithm

In [42]:
model = SageMakerModelRunner(
    endpoint_name = 'mixtral-8bit-20231213-122707',
    content_template =  '{"inputs": $prompt, "parameters": {"max_new_tokens": 50}}',
    output="[0].generated_text"
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [43]:
# Evaluate factual_knowledge
eval_algorithm_config = FactualKnowledgeConfig("<OR>")
eval_algo = FactualKnowledge(eval_algorithm_config)

In [44]:
# Custom dataset
dataset_config = DataConfig(
        dataset_name="TREX",
        dataset_uri="trex_sample.jsonl",
        dataset_mime_type="application/jsonlines",
        model_input_location="question",
        target_output_location="answers",
        category_location="knowledge_category",
    )

In [45]:
# Evaluate model with fmeval built-in dataset
eval_outputs = eval_algo.evaluate(model=model, dataset_config=dataset_config, prompt_template="$feature", save=True)

2023-12-13 15:29:12,028	INFO read_api.py:406 -- To satisfy the requested parallelism of 4, each read task output is split into 4 smaller blocks.


Read progress 0:   0%|          | 0/1 [00:00<?, ?it/s]

Read progress 0:   0%|          | 0/1 [00:00<?, ?it/s]

2023-12-13 15:29:12,149	INFO streaming_executor.py:93 -- Executing DAG InputDataBuffer[Input] -> AllToAllOperator[Repartition]
2023-12-13 15:29:12,151	INFO streaming_executor.py:94 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=True, actor_locality_enabled=True, verbose_progress=False)
2023-12-13 15:29:12,152	INFO streaming_executor.py:96 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_options.verbose_progress = True`


- Repartition 1:   0%|          | 0/5 [00:00<?, ?it/s]

...

Sort Sample 0:   0%|          | 0/5 [00:00<?, ?it/s]

2023-12-13 16:30:03,741	INFO streaming_executor.py:93 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[Map(_generate_eval_scores)->Map(<lambda>)]
2023-12-13 16:30:03,747	INFO streaming_executor.py:94 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=True, actor_locality_enabled=True, verbose_progress=False)
2023-12-13 16:30:03,753	INFO streaming_executor.py:96 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_options.verbose_progress = True`


Running 0:   0%|          | 0/5 [00:00<?, ?it/s]

2023-12-13 16:30:04,048	INFO streaming_executor.py:93 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[Map(_generate_eval_scores)->Map(<lambda>)]
2023-12-13 16:30:04,051	INFO streaming_executor.py:94 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=True, actor_locality_enabled=True, verbose_progress=False)
2023-12-13 16:30:04,055	INFO streaming_executor.py:96 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_options.verbose_progress = True`


Running 0:   0%|          | 0/5 [00:00<?, ?it/s]

In [46]:
# Show Evaluation outputs
print(eval_outputs)

[EvalOutput(eval_name='factual_knowledge', dataset_name='TREX', dataset_scores=[EvalScore(name='factual_knowledge', value=0.7661691542288557)], prompt_template='$feature', category_scores=[CategoryScore(name='Capitals', scores=[EvalScore(name='factual_knowledge', value=0.88)]), CategoryScore(name='Subsidiary', scores=[EvalScore(name='factual_knowledge', value=0.6534653465346535)])], output_path='/tmp/eval_results/factual_knowledge_TREX.jsonl', error=None)]


In [48]:
import json
print(json.dumps(eval_outputs, default=vars, indent=4))

[
    {
        "eval_name": "factual_knowledge",
        "dataset_name": "TREX",
        "dataset_scores": [
            {
                "name": "factual_knowledge",
                "value": 0.7661691542288557
            }
        ],
        "prompt_template": "$feature",
        "category_scores": [
            {
                "name": "Capitals",
                "scores": [
                    {
                        "name": "factual_knowledge",
                        "value": 0.88
                    }
                ]
            },
            {
                "name": "Subsidiary",
                "scores": [
                    {
                        "name": "factual_knowledge",
                        "value": 0.6534653465346535
                    }
                ]
            }
        ],
        "output_path": "/tmp/eval_results/factual_knowledge_TREX.jsonl",
        "error": null
    }
]


In [50]:
!cat /tmp/eval_results/factual_knowledge_TREX.jsonl

{"model_input": "Primark is a subsidiary of", "model_output": "Primark is a subsidiary of Associated British Foods. It is a clothing retailer that sells a wide range of clothing, accessories, footwear, homeware, beauty products, and confectionery. Primark has over 370 stores in 12", "target_output": "Associated British Foods", "category": "Subsidiary", "prompt": "Primark is a subsidiary of", "scores": [{"name": "factual_knowledge", "value": 1}]}
{"model_input": "Chartered Industries is a subsidiary of", "model_output": "Chartered Industries is a subsidiary of the Chartered Group of Companies, which is a conglomerate of companies with interests in various sectors of the economy.\n\nChartered Industries is a leading manufacturer of high quality packaging materials in Nigeria. The company was established in", "target_output": "ST Engineering", "category": "Subsidiary", "prompt": "Chartered Industries is a subsidiary of", "scores": [{"name": "factual_knowledge", "value": 0}]}
{"model_input